# Customizing channels

In [ ]:
import sinaps as sn

In [ ]:
import numpy as np

To implement a channel C, it is necessary to implement  :
 - a static method  C._I(V,t,**st_vars,**params) returning the net  current towards inside
  ([A] for point channel and [pA/μm2] for density channels ) of the mechanism with
 V the voltage (:array) and **st_vars the state variable of the mechanism (:array)
 - If there is state variables a static method C._dS(V,t,**st_vars,**params) returning a
  tuple with the differential of each state variable
   and a function S0 returning a tuple with the initial value for each state
   variable
   
   C._J for spevies
"""

you need to subclass Channel Class :

in this example we will create to additionals channels

In [ ]:
class AMPAR(sn.Channel):
    """Point channel with a AMPAr-type current starting at time t0, 

    """
    param_names = ('gampa','tampa1','tampa2','V_ampa','t0')

    def __init__(self,t0,gampa=0.02,tampa1=0.3,tampa2=3,V_ampa=70):
        """Point channel with a AMPAr-type current starting at time t0 [pA]
            t0: start of the current [ms]
            gampa: max conductance of Ampar []nS]
            tampa1: Ampar time constant [ms]
            tampa2: Ampar time constant [ms]
            V_ampa: Ampar Nernst potential [mV]
        """
        self.params={'t0' : t0,
                     'gampa' : gampa,
                     'tampa1' : tampa1,
                     'tampa2' : tampa2,
                     'V_ampa': V_ampa,
                    }

    @staticmethod
    def _I(V,t,
           t0,gampa,tampa1,tampa2,V_ampa):
        return ((t <= t0+20) & (t >= t0)) * (-gampa*(1-np.exp(-t/tampa1))*np.exp(-t/tampa2)*(V-V_ampa))
    
    @staticmethod
    def _J(ion,V,t,
           t0,gampa,tampa1,tampa2,V_ampa):
        """
        Return the flux of ion [aM/ms/um2] of the mechanism towards inside
        """
        if ion is sn.Species.Ca:
            return ((t <= t0+20) & (t >= t0)) * np.maximum(-gampa*(1-np.exp(-t/tampa1))*np.exp(-t/tampa2)*(V-V_ampa) /96.48533132838746/2*0.014,0)
        else:
            return 0 * V

In [ ]:
class NMDAR(sn.Channel):
    """Point channel with a NMDAr-type current starting at time t0, 
voltage-dependent flow of sodium (Na+) and small amounts of calcium (Ca2+) ions into the cell and potassium (K+) out of the cell.
    """
    param_names = ('t0','gnmda','tnmda1','tnmda2','V_nmda')

    def __init__(self,t0,gnmda=0.02,tnmda1=11.5,tnmda2=0.67,V_nmda=75):
        """Point channel with a AMPAr-type current starting at time t0 [pA]
            t0: start of the current [ms]
            gnmda: max conductance of NMDAr [nS]
            tnmda1: NMDAr time constant [ms]
            tnmda2: NMDAr time constant [ms]
            V_nmda: NMDAr Nernst potential [mV]
        """
        self.params={'t0' : t0,
                     'gnmda' : gnmda,
                     'tnmda1' : tnmda1,
                     'tnmda2' : tnmda2,
                     'V_nmda': V_nmda,
                    }

    @staticmethod
    def _I(V,t,
           t0,gnmda,tnmda1,tnmda2,V_nmda):
        return -((t <= t0+50) & (t >= t0))*gnmda*(np.exp(-(t-t0)/tnmda1)-np.exp(-(t-t0)/tnmda2))/(1+0.33*2*np.exp(-0.06*(V-65)))*(V-V_nmda)

    
    @staticmethod
    def _J(ion,V,t,
           t0,gnmda,tnmda1,tnmda2,V_nmda):
        """
        Return the flux of ion [aM/ms/um2] of the mechanism towards inside
        """
        if ion is sn.Species.Ca:
            return ((t <= t0+50) & (t >= t0)) *np.maximum(-gnmda*(np.exp(-(t-t0)/tnmda1)-np.exp(-(t-t0)/tnmda2))/(1+0.33*2*np.exp(-0.06*(V-65)))*(V-V_nmda)/96.48533132838746/2*0.15,0)
        else:
            return 0 *V

### Setting up the channels

In [ ]:
nrn = sn.Neuron([(0,1),(1,2),(1,3),(3,4)])

In [ ]:
nrn[0]

In [ ]:
nrn[:].clear_channels()
nrn[:].add_channel(sn.channels.Hodgkin_Huxley())
nrn[:].add_channel(sn.channels.Hodgkin_Huxley_Ca())
nrn[0].add_channel(NMDAR(0.5,gnmda=2),0) #

### Running the simu

In [ ]:
# Initialisation of the simulation
sim=sn.Simulation(nrn,dx=100)

In [ ]:
# Runing the simulation
sim.run((0,1000))

### Plots

In [ ]:
# Plotting the potential
sim.plot()

In [ ]:
# Extracting some sections
sim['Section0060'].plot() * sim['Section0000'].plot()

### Chemical reactions

In [ ]:
# Clearing previous reactions
N.reactions=[]
# Adding Ca + BF <-> BFB (BF = GCamp, BFB = GCamp-Ca)
N.add_reaction(
    {Ca:1,
     BF:1},
    {BFB:1},
    k1=12.3,
    k2=0.002)
# Calcium extrusion
N.add_reaction(
    {Ca:1},
    {},
    k1=0.003,k2=0)
#Calcium initial concentration
for s in N:
    s.C0[Ca]=10**(-7)
#Buffer initial concentration
for s in N:
    s.C0[BF]=10**(-6)

In [ ]:
# Running the chemical reactions part   
sim.run_diff(max_step=1)

### Plots

In [ ]:
# Plot of the Calcium concentration
sim[:].plot.C(Ca)

In [ ]:
# Plot of GCamp bound with Ca
sim[:].plot.C(BFB)

In [ ]:
# 2D plots, non linear in time (plotting against dt in the simulations) Highlits time periods with activity. Careful: the graph don't corresponds in the x axis, as the simulation for the potential and for the chemical reactions are separated.
( graph2D(sim.V[dfs]).opts(title='Potential',    **common_opts)
+ graph2D(sim.C[Ca][dfs]).opts(title='Calcium',      **common_opts)
+ graph2D(sim.C[BFB][dfs]).opts(title='Bound Calcium',**common_opts)
    )

In [ ]:
# 2D graph linear in time
(graph2Dlinear(sim.V[dfs]).opts(title='Potential', **common_opts)
+ graph2Dlinear(sim.C[Ca][dfs]).opts(title='Calcium',      **common_opts)
+ graph2Dlinear(sim.C[BFB][dfs]).opts(title='Bound Calcium',**common_opts)
    )

In [ ]:
# 2D graph linear in time, extracting the first 6 ms
(graph2Dlinear(sim.V[0:6][dfs]).opts(title='Potential', **common_opts)
+ graph2Dlinear(sim.C[0:6][Ca][dfs]).opts(title='Calcium',      **common_opts)
+ graph2Dlinear(sim.C[0:6][BFB][dfs]).opts(title='Bound Calcium',**common_opts)
    )

In [ ]:
graph2Dlinear(sim.C[BFB][dfs]).opts(title='Bound Calcium',**common_opts)

In [ ]:
Test=sim.C[BFB][:].iloc
N[:].name
for name in N[:].name:
    sim.C[BFB][:].iloc[:,0]/N[name].a
    sim.C[BFB][:].iloc[:,1]/N[name].a

In [ ]:
Test=sim.C[BFB].values


In [ ]:
sim.C[BFB][:].iloc[:,1]

## Currents

In [ ]:
# Currents in the simulation
[ch.__name__ for ch in sim.channels]

In [ ]:
I(sim,'Hodgkin_Huxley')['Section0484'].iloc[0:550,0].hvplot()

In [ ]:
I(sim,'Hodgkin_Huxley_Ca')['Section0484'].iloc[0:600,0].hvplot()

In [ ]:
I(sim,'NMDAR')['Section0236'].clip(0).iloc[0:600,1].hvplot()

In [ ]:
graph2Dlinear(I(sim,'Hodgkin_Huxley')[dfs][0:4]).opts(**common_opts,title='HH Current')

In [ ]:
graph2Dlinear(I(sim,'NMDAR')[dfs][0:4].clip(0)).opts(**common_opts,title='NMDA Current')